In [16]:
!pip install lyricsgenius

In [17]:
GENUIS_API_KEY = '83SA8j6SDnJDK5g2bYnHWjAMVvj4G4zLl1SyPQ-H30Mv-o8VHlAtUHtD2TZ9Z7jW'


In [18]:
import pandas as pd


data = pd.read_csv("spotify/data/data.csv")
genre_data = pd.read_csv('spotify/data/data_by_genres.csv')
year_data = pd.read_csv('spotify/data/data_by_year.csv')


In [2]:
import os
import pandas as pd

# Setting the base directory using list of directory names
base_dir = "data"

# Building paths by further extending the base directory
data_path = os.path.join(base_dir, "data.csv")
genre_data_path = os.path.join(base_dir, "data_by_genres.csv")
year_data_path = os.path.join(base_dir, "data_by_year.csv")

# Reading the data using pandas
data = pd.read_csv(data_path)
genre_data = pd.read_csv(genre_data_path)
year_data = pd.read_csv(year_data_path)


In [19]:
data.head()


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [20]:

import lyricsgenius as lg
genius = lg.Genius(GENUIS_API_KEY)
genius = lg.Genius(GENUIS_API_KEY, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
genius = lg.Genius(GENUIS_API_KEY, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)


In [22]:
# #Searches all of "Pop Smoke"'s songs.
# artist = genius.search_artist("Pop Smoke")
# #Searches 5 of "Pop Smoke"'s songs.
# artist = genius.search_artist("Pop Smoke",max_songs=5)
#Searches 5 of "Pop Smoke"'s songs via its Title.
artist = genius.search_artist("PSYSmoke",max_songs=5,sort="title")
# Sorts = popularity release_date title
song = genius.search_song("Dior", artist.name)
print(song)

print(song.lyrics)


SyntaxError: unmatched ')' (3081229146.py, line 8)

In [4]:
!pip install spotipy

  Obtaining dependency information for spotipy from https://files.pythonhosted.org/packages/ea/35/304e456a471128aa4a776243558f43aee3444731ef8fc9bc8c351fddfdd8/spotipy-2.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for redis>=3.5.3 from https://files.pythonhosted.org/packages/c5/d1/19a9c76811757684a0f74adc25765c8a901d67f9f6472ac9d57c844a23c8/redis-5.0.8-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 4.9 MB/s eta 0:00:00a 0:00:01


In [6]:
# spotify 


import spotipy
client_id = '10cc8ee290404da9ab9d7b061d526193'
client_secret = '0dc9cb56d8bc4454afa1ddbe82a7301d'

from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# check health 
sp.trace = False
track = sp.track('7qiZfU4dY1lWllzX7mPBI3')
print(track['name'])
# Now you have the access token to make requests to the Spotify API


Shape of You


In [7]:
# get user playlist
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

client_id = '10cc8ee290404da9ab9d7b061d526193'
client_secret = '0dc9cb56d8bc4454afa1ddbe82a7301d'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlists = sp.user_playlists('spotify')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

   1 spotify:playlist:37i9dQZF1DXcBWIGoYBM5M Today’s Top Hits
   2 spotify:playlist:37i9dQZF1DX0XUsuxWHRQd RapCaviar
   3 spotify:playlist:37i9dQZF1DX1lVhptIYRda Hot Country
   4 spotify:playlist:37i9dQZF1DX10zKzsJ2jva Viva Latino
   5 spotify:playlist:37i9dQZF1DX4JAvHpjipBk New Music Friday
   6 spotify:playlist:37i9dQZF1DX4sWSpwq3LiO Peaceful Piano
   7 spotify:playlist:37i9dQZF1DX4SBhb3fqCJd RNB X
   8 spotify:playlist:37i9dQZF1DWXRqgorJj26U Rock Classics
   9 spotify:playlist:37i9dQZF1DX4dyzvuaRJ0n mint
  10 spotify:playlist:37i9dQZF1DXcF6B6QPhFDv MARROW
  11 spotify:playlist:37i9dQZF1DWXJfnUiYjUKT New Music Friday
  12 spotify:playlist:37i9dQZF1DXcRXFNfZr7Tp just hits
  13 spotify:playlist:37i9dQZF1DX4o1oenSJRJd All Out 2000s
  14 spotify:playlist:37i9dQZF1DXbTxeAdrVG2l All Out 90s
  15 spotify:playlist:37i9dQZF1DX4UtSsGT1Sbe All Out 80s
  16 spotify:playlist:37i9dQZF1DWTJ7xPn4vNaz All Out 70s
  17 spotify:playlist:37i9dQZF1DXaKIA8E7WcJj All Out 60s
  18 spotify:playlist:37i9dQZF1